# Obter dados no drive

In [ ]:
!pip install tensorflow_model_optimization

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from google.colab import drive
import pandas as pd
import random
import os
import time
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from glob import glob
from sklearn.model_selection import train_test_split
import numpy as np
from keras import layers, regularizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import ParameterGrid


In [ ]:
drive.mount('/content/drive')

# Carregamento de dados - Sem Data Augmentation


In [ ]:
images = []
masks = []

for path in glob('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/cropped_images/*/*'):
  images.append(path + '/image.tif')
  masks.append(path + '/mask.png')

In [ ]:
# Função para carregar e pré-processar uma imagem e sua máscara
def load_and_preprocess_image(image_path, mask_path, target_size):

    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image) / 255.0  # Normalização entre 0 e 1

    mask = load_img(mask_path, target_size=target_size, color_mode='grayscale')
    mask = img_to_array(mask) / 255.0  # Normalização entre 0 e 1

    return image, mask

In [ ]:
with tf.device('/gpu:0'):
    # Lista para armazenar imagens e máscaras pré-processadas
    images_processed = []
    masks_processed = []

    # Carregar e pré-processar todas as imagens e máscaras
    for img_path, mask_path in zip(images, masks):
        img, mask = load_and_preprocess_image(img_path, mask_path, target_size=(256, 256))
        images_processed.append(img)
        masks_processed.append(mask)

    # Converter para arrays numpy
    images_processed = np.array(images_processed)
    masks_processed = np.array(masks_processed)


In [ ]:
images_processed

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images_processed, masks_processed, test_size=0.3, random_state=42)

# Definição de parâmetros e modelo

In [ ]:
class CyclicLR(Callback):
    def __init__(self, base_lr=1e-4, max_lr=1e-3, step_size=2000., mode='triangular'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.iterations = 0
        self.history = {}

    def clr(self):
        cycle = np.floor(1 + self.iterations / (2 * self.step_size))
        x = np.abs(self.iterations / self.step_size - 2 * cycle + 1)
        lr = self.base_lr + (self.max_lr - self.base_lr) * max(0, (1 - x))
        if self.mode == 'triangular2':
            lr = lr / float(2 ** (cycle - 1))
        elif self.mode == 'exp_range':
            lr = lr * (0.999 ** self.iterations)
        return lr

    def on_train_begin(self, logs=None):
        logs = logs or {}
        tf.keras.backend.set_value(self.model.optimizer.lr, self.base_lr)

    def on_batch_end(self, batch, logs=None):
        self.iterations += 1
        lr = self.clr()
        tf.keras.backend.set_value(self.model.optimizer.lr, lr)
        self.history.setdefault('lr', []).append(lr)
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

# Função de callbacks
def get_callbacks():
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
    clr = CyclicLR(base_lr=1e-4, max_lr=1e-3, step_size=2000., mode='triangular2')
    #pruning = sparsity.UpdatePruningStep()
    #return [pruning, early_stopping, reduce_lr, clr]
    return [early_stopping, reduce_lr, clr]

In [ ]:
# Função para calcular a sigmoide e converter para 0 ou 1 o output
class ThresholdLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.where(inputs < 0.5, 0.0, 1.0)

# Função para calcular o Dice Coefficient
def dice_coefficient(y_train, y_val):
    smooth = 1e-6
    intersection = tf.reduce_sum(y_train * y_val)
    dice_coefficient = (2. * intersection + smooth) / (tf.reduce_sum(y_train) + tf.reduce_sum(y_val) + smooth)
    return dice_coefficient

# Função de perda de Dice
def dice_loss(y_train, y_val):
    return 1 - dice_coefficient(y_train, y_val)

# Função para calcular a penalidade adicional
def penalty_loss(y_train, y_val, penalty_weight):
    # Calcular a penalidade considerando a diferença entre y_train e y_val
    penalty = tf.reduce_sum(tf.abs(y_train - y_val))
    # Multiplicar a penalidade pelo peso da penalidade
    weighted_penalty = penalty_weight * penalty
    return weighted_penalty

# Função de perda combinada
def combined_loss(y_train, y_val, alpha, beta, gamma, penalty_weight):
    # Perda padrão (por exemplo, perda de entropia cruzada binária)
    standard_loss = tf.keras.losses.binary_crossentropy(y_train, y_val)
    dice = dice_loss(y_train, y_val) # Dice Loss
    penalty = penalty_loss(y_train, y_val, penalty_weight) # Penalidade adicional
    # Perda total = perda padrão + penalidade + Dice Loss
    total_loss = alpha * standard_loss + beta * dice + gamma * penalty
    return total_loss

# Métrica de acurácia customizada
def custom_accuracy(y_train, y_val):
    # Calcular a acurácia considerando uma tolerância de 0.5 na predição
    y_val_binary = tf.round(y_val)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(y_train, y_val_binary), tf.float32))
    return accuracy

In [ ]:
# Carregar o modelo MobileNetV2 pré-treinado
pre_trained_model = MobileNetV2(weights='imagenet', include_top=False)

# Visualizar a estrutura do modelo
pre_trained_model.summary()

In [ ]:
class UNet:
    def __init__(self, input_shape, num_filters, kernel_size, dropout_rate, val_reg):
        self.input_shape = input_shape
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.dropout_rate = dropout_rate
        self.val_reg = val_reg

    def build_model(self):
        inputs = tf.keras.Input(shape=self.input_shape)
        reg = regularizers.L2(self.val_reg)

        # Encoder (contraction path)
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=inputs)
        base_model.trainable = False

        # Extract specific layers for connections
        conv2 = base_model.get_layer('block_1_expand').output
        conv3 = base_model.get_layer('block_3_expand').output
        decoded = base_model.get_layer('block_6_expand').output

        # Decoder (expansion path)
        up5 = layers.Conv2DTranspose(self.num_filters[2], (2, 2), strides=(2, 2), padding='same')(decoded)
        merge5 = layers.concatenate([conv3, up5], axis=3)
        conv5 = layers.Conv2D(self.num_filters[2], self.kernel_size, activation='relu', padding='same')(merge5)
        conv5 = layers.Conv2D(self.num_filters[2], self.kernel_size, activation='relu', padding='same')(conv5)

        up6 = layers.Conv2DTranspose(self.num_filters[1], (2, 2), strides=(2, 2), padding='same')(conv5)
        merge6 = layers.concatenate([conv2, up6], axis=3)
        conv6 = layers.Conv2D(self.num_filters[1], self.kernel_size, activation='relu', padding='same')(merge6)
        conv6 = layers.Conv2D(self.num_filters[1], self.kernel_size, activation='relu', padding='same')(conv6)

        up7 = layers.Conv2DTranspose(self.num_filters[0], (2, 2), strides=(2, 2), padding='same')(conv6)
        conv7 = layers.Conv2D(self.num_filters[0], self.kernel_size, activation='relu', padding='same')(up7)
        conv7 = layers.Conv2D(self.num_filters[0], self.kernel_size, activation='relu', padding='same')(conv7)

        outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(conv7)  # Saída com um canal (máscara binária)

        threshold_output = ThresholdLayer()(outputs)

        model = tf.keras.Model(inputs=inputs, outputs=outputs)

        return model

    def compile_and_train(self, X_train, y_train, X_val, y_val, max_epochs, batch_size, alpha, beta, gamma, penalty_weight):
        model = self.build_model()
        callbacks = get_callbacks()

        # Compilar o modelo
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                      loss=lambda y_train, y_val: combined_loss(y_train, y_val, alpha, beta, gamma, penalty_weight),
                      metrics=[custom_accuracy])

        #model.summary()

        start_time = time.time()

        # Treinar o modelo
        H = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                      epochs=max_epochs, batch_size=batch_size, callbacks=callbacks)

        # Salvar o tempo de treino
        training_time_gpu = time.time() - start_time

        return H, training_time_gpu, model


In [ ]:
def evaluate_model(model, X_val, y_val, H, training_time_gpu, max_epochs):
    plt.style.use("ggplot")
    plt.figure(figsize=(12, 10))

    start_time = time.time()

    results = model.evaluate(X_val, y_val)

    inference_time_gpu = time.time() - start_time

    print("Test Loss:", results[0])
    print("Test Accuracy:", results[1])

    # Prever máscaras usando o modelo
    predicted_masks = model.predict(X_val)

    # Obter métricas de precisão e perda do treinamento
    acc = H.history['custom_accuracy']
    val_acc = H.history['val_custom_accuracy']
    loss = H.history['loss']
    val_loss = H.history['val_loss']

    epochs = range(1, len(acc) + 1)

    # Plotar precisão do conjunto
    plt.subplot(2, 2, 3)
    plt.plot(epochs, acc, 'r', label='Precisão do Conjunto de Treino')
    plt.plot(epochs, val_acc, 'b', label='Precisão do Conjunto de Validação')
    plt.title('Precisão do Conjunto de Treino e Validação')
    plt.xlabel('Épocas')
    plt.ylabel('Precisão')
    plt.legend()

    # Plotar perda do conjunto
    plt.subplot(2, 2, 4)
    plt.plot(epochs, loss, 'r', label='Perda do Conjunto de Treino')
    plt.plot(epochs, val_loss, 'b', label='Perda do Conjunto de Validação')
    plt.title('Perda do Conjunto de Treino e Validação')
    plt.xlabel('Épocas')
    plt.ylabel('Perda')
    plt.legend()

    plt.tight_layout()
    plt.show()

    print('Tempo de treino (segundos): ', training_time_gpu)
    print('Tempo de treino por época (segundos): ', training_time_gpu / max_epochs)
    print('Tempo de inferência (segundos): ', inference_time_gpu)

    return predicted_masks, inference_time_gpu

In [ ]:
# Grid de parâmetros para busca
param_grid = {
    'alpha': [0.1, 1.0, 10.0],
    'beta': [0.1, 1.0],
    'gamma': [0.0001, 0.001, 0.01],
    'penalty_weight': [0.0001, 0.001]
}

grid = ParameterGrid(param_grid)

max_epochs = 30
results = []

test_counter = 1
H_dict = {}
training_time_dict = {}
model_dict = {}

for params in grid:
    print(f'Teste {test_counter}: Testando com parâmetros: {params}')
    model = UNet(input_shape=(256, 256, 3),
                 num_filters=(16, 32, 64, 128),
                 kernel_size = 3,
                 dropout_rate=0.1,
                 val_reg=0.01)

    # Compilar e treinar o modelo com os parâmetros do grid search
    H, training_time_gpu, model = model.compile_and_train(X_train, y_train, X_val, y_val,
                                                          max_epochs=max_epochs,
                                                          batch_size=16,
                                                          alpha=params['alpha'],
                                                          beta=params['beta'],
                                                          gamma=params['gamma'],
                                                          penalty_weight=params['penalty_weight'])

    H_dict[f'H{test_counter}'] = H
    training_time_dict[f'training_time_gpu{test_counter}'] = training_time_gpu
    model_dict[f'model{test_counter}'] = model

    predicted_masks, inference_time_gpu = evaluate_model(model, X_val, y_val, H, training_time_gpu, max_epochs=max_epochs)
    val_loss = H.history['val_loss'][-1]
    val_acc = H.history['val_custom_accuracy'][-1]
    results.append((params, val_loss, val_acc))
    test_counter += 1

In [ ]:
# Print the results
for result in results:
    print(f"Params: {result[0]}, Val Loss: {result[1]}, Val Accuracy: {result[2]}")

In [ ]:
# Convertemos os resultados em um DataFrame para facilitar a visualização
results_df = pd.DataFrame(results, columns=['params', 'val_loss', 'val_custom_accuracy'])

# Expandimos os parâmetros para colunas separadas
params_df = pd.json_normalize(results_df['params'])
results_df = pd.concat([params_df, results_df[['val_loss', 'val_custom_accuracy']]], axis=1)
print(results_df)

# Plotar os resultados
plt.figure(figsize=(20, 10))

# Plotar perda de validação para cada parâmetro
for i, param in enumerate(['alpha', 'beta', 'gamma', 'penalty_weight']):
    plt.subplot(2, 2, i+1)
    for key, grp in results_df.groupby([p for p in ['alpha', 'beta', 'gamma', 'penalty_weight'] if p != param]):
        plt.plot(grp[param], grp['val_loss'], marker='o', label=f'Fixando {key}')
    plt.xlabel(param)
    plt.ylabel('Perda de Validação')
    plt.title(f'Impacto de {param} na Perda de Validação')
    plt.xscale('log')
    plt.legend()

plt.tight_layout()
plt.show()

plt.figure(figsize=(20, 10))

# Plotar acurácia de validação para cada parâmetro
for i, param in enumerate(['alpha', 'beta', 'gamma', 'penalty_weight']):
    plt.subplot(2, 2, i+1)
    for key, grp in results_df.groupby([p for p in ['alpha', 'beta', 'gamma', 'penalty_weight'] if p != param]):
        plt.plot(grp[param], grp['val_custom_accuracy'], marker='o', label=f'Fixando {key}')
    plt.xlabel(param)
    plt.ylabel('Acurácia de Validação')
    plt.title(f'Impacto de {param} na Acurácia de Validação')
    plt.xscale('log')
    plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
results_df

In [ ]:
max_epochs = 30

print(f'Teste sem hyperparâmetros: ')
model = UNet(input_shape=(256, 256, 3),
                num_filters=(16, 32, 64, 128),
                kernel_size = 3,
                dropout_rate=0.1,
                val_reg=0.01)

# Compilar e treinar o modelo com os parâmetros do grid search
H_clean, training_time_gpu_clean, model_clean = model.compile_and_train(X_train, y_train, X_val, y_val,
                                                        max_epochs=max_epochs,
                                                        batch_size=16,
                                                        alpha=1.0,
                                                        beta=1.0,
                                                        gamma=1.0,
                                                        penalty_weight=0.001)

predicted_masks, inference_time_gpu = evaluate_model(model_clean, X_val, y_val, H_clean, training_time_gpu_clean, max_epochs=max_epochs)


In [ ]:
# max_epochs = 30

# print(f'Teste sem hyperparâmetros: ')
# model = UNet(input_shape=(256, 256, 3),
#                 num_filters=(16, 32, 64, 128),
#                 kernel_size = 3,
#                 dropout_rate=0.1,
#                 val_reg=0.01)

# # Compilar e treinar o modelo com os parâmetros do grid search
# H_clean, training_time_gpu_clean, model_clean = model.compile_and_train(X_train, y_train, X_val, y_val,
#                                                         max_epochs=max_epochs,
#                                                         batch_size=16,
#                                                         alpha=10.0,
#                                                         beta=1.0,
#                                                         gamma=0.0001,
#                                                         penalty_weight=0.001)

# predicted_masks, inference_time_gpu = evaluate_model(model_clean, X_val, y_val, H_clean, training_time_gpu_clean, max_epochs=max_epochs)

## Amostragem das máscaras e IoU

In [ ]:
with tf.device('/gpu:0'):
    # Gerar as saídas do modelo para um conjunto de entradas de teste
    saidas_modelo = model_dict[f'model{32}'].predict(X_val)

    # Iterar sobre cada saída do modelo
    for i in range(len(X_val)):
        # Obter a entrada correspondente e a saída real
        img_entrada = X_val[i]
        img_saida_real = y_val[i]

        # Obter a saída gerada pelo modelo
        img_saida_modelo = saidas_modelo[i]

        # Mostrar as imagens
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 3, 1)
        plt.imshow(img_entrada.squeeze(), cmap='gray')
        plt.title('Entrada')
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(img_saida_real.squeeze(), cmap='gray')
        plt.title('Saída Esperada')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(img_saida_modelo.squeeze(), cmap='gray')
        plt.title('Saída do Modelo - GPU')
        plt.axis('off')

        plt.show()

In [ ]:
with tf.device('/gpu:0'):
    # Métricas do parceiro de Projeto:

    # Lista para armazenar os scores de IoU
    iou_scores = []
    # Calcular IoUs e determinar predições corretas
    correct_predictions = 0
    iou_threshold = 0.5
    for mask, result in zip(y_val, img_saida_modelo):
        intersection = np.logical_and(mask, result)
        union = np.logical_or(mask, result)
        iou_score = np.sum(intersection) / np.sum(union) if np.sum(union) != 0 else 0
        iou_scores.append(iou_score)
        # Verificar se a predição é considerada correta (IoU >= threshold)
        if iou_score >= iou_threshold:
            correct_predictions += 1
        print('IoU é: ' + str(iou_score))
    # Calcular a média dos IoUs
    iou_mean = np.mean(iou_scores)
    print('Média dos IoU - GPU:', iou_mean)
    # Calcular Coverage Ratio (CovR)
    total_predictions = len(iou_scores)
    covr = correct_predictions / total_predictions if total_predictions > 0 else 0
    print('Coverage Ratio (CovR) - GPU:', covr)

À princípio, os melhores testes encontrados com menor indício de overfitting e com as maiores acurácias foram:

Teste 32: Testando com parâmetros: {'alpha': 10.0, 'beta': 1.0, 'gamma': 0.0001, 'penalty_weight': 0.001}

Teste 26: Testando com parâmetros: {'alpha': 10.0, 'beta': 0.1, 'gamma': 0.0001, 'penalty_weight': 0.001}

Teste 25: Testando com parâmetros: {'alpha': 10.0, 'beta': 0.1, 'gamma': 0.0001, 'penalty_weight': 0.0001}

Teste 20: Testando com parâmetros: {'alpha': 1.0, 'beta': 1.0, 'gamma': 0.0001, 'penalty_weight': 0.001}

Teste 17: Testando com parâmetros: {'alpha': 1.0, 'beta': 0.1, 'gamma': 0.01, 'penalty_weight': 0.0001}

Teste 13: Testando com parâmetros: {'alpha': 1.0, 'beta': 0.1, 'gamma': 0.0001, 'penalty_weight': 0.0001}

Teste 11: Testando com parâmetros: {'alpha': 0.1, 'beta': 1.0, 'gamma': 0.01, 'penalty_weight': 0.0001}

Teste 5: Testando com parâmetros: {'alpha': 0.1, 'beta': 0.1, 'gamma': 0.01, 'penalty_weight': 0.0001}
